# Introduction to Scipy: Fitting data

We have talked about the Numpy and Matplotlib libraries, but there is a third library that is invaluable for Scientific Analysis: [Scipy](http://www.scipy.org). Scipy is basically a very large library of functions that you can use for scientific analysis. A good place to start to find out about the top-level scientific functionality in Scipy is the [Documentation](http://docs.scipy.org/doc/scipy/reference/).

Examples of the functionality include:

* Integration (scipy.integrate)
* Optimization/Fitting (scipy.optimize)
* Interpolation (scipy.interpolate)
* Fourier Transforms (scipy.fftpack)
* Signal Processing (scipy.signal)
* Linear Algebra (scipy.linalg)
* Spatial data structures and algorithms (scipy.spatial)
* Statistics (scipy.stats)
* Multi-dimensional image processing (scipy.ndimage)

and so on.

In this section, we will take a look at how to fit models to data. When analyzing scientific data, fitting models to data allows us to determine the parameters of a physical system (assuming the model is correct).

There are a number of routines in Scipy to help with fitting, but we will use the simplest one, ``curve_fit``, which is imported as follows:

In [ ]:
import numpy as np
from scipy.optimize import curve_fit

The full documentation for the ``curve_fit`` is available [here](http://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html#scipy.optimize.curve_fit), and we will look at a simple example here, which involves fitting a straight line to a dataset.

We first create a fake dataset with some random noise:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
x = np.random.uniform(0., 100., 100)
y = 3. * x + 2. + np.random.normal(0., 10., 100)
plt.plot(x, y, '.')

Let's now imagine that this is real data, and we want to determine the slope and intercept of the best-fit line to the data. We start off by definining a function representing the model:

In [ ]:
def line(x, a, b):
    return a * x + b

The arguments to the function should be ``x``, followed by the parameters. We can now call ``curve_fit`` to find the best-fit parameters using a least-squares fit:

In [ ]:
popt, pcov = curve_fit(line, x, y)

The ``curve_fit`` function returns two items, which we call ``popt`` and ``pcov``. The ``popt`` argument are the best-fit paramters for ``a`` and ``b``:

In [ ]:
popt
plt.plot(x, y, '.')
plt.plot(x, line(x, popt[0], popt[1]), 'r-')

which is close to the initial values of ``3`` and ``2`` used in the definition of ``y``.

The reason the values are not exact is because there are only a limited number of random samples, so the best-fit slope is not going to be exactly those used in the definition of ``y``. The ``pcov`` variable contains the *covariance* matrix, which indicates the uncertainties and correlations between parameters. This is mostly useful when the data has uncertainties.

In [ ]:
print(popt,pcov)

Let's now try and fit the data assuming each point has a vertical error (standard deviation) of +/-10:

In [ ]:
e = np.repeat(10., 100)
plt.plot(x, y, '.')
plt.errorbar(x, y, yerr=e, fmt="none")

In [ ]:
popt, pcov = curve_fit(line, x, y, sigma=e)

In [ ]:
popt

Now ``pcov`` will contain the true variance and covariance of the parameters, so that the best-fit parameters are:

In [ ]:
print("a =", popt[0], "+/-", pcov[0,0]**0.5)
print("b =", popt[1], "+/-", pcov[1,1]**0.5)

We can now plot the best-fit line:

In [ ]:
plt.errorbar(x, y, yerr=e, fmt="none")
xfine = np.linspace(0., 100., 100)  # define values to plot the function for
plt.plot(xfine, line(xfine, popt[0], popt[1]), 'r-')

You should now be able to fit simple models to datasets! Note that for more complex models, more sophisticated techniques may be required for fitting, but ``curve_fit`` will be good enough for most simple cases.

Note that there is a way to simplify the call to the function with the best-fit parameters, which is:

    line(x, *popt)

The * notation will expand a list of values into the arguments of the function. This is useful if your function has more than one or two parameters. Hence, you can do:

In [ ]:
plt.errorbar(x, y, yerr=e, fmt="none")
plt.plot(xfine, line(xfine, *popt), 'r-')

**Important Note:**  ``curve_fit`` by default uses sigma just as weights, i.e., as hints how important a point should be in the least-squares computation.  To compute the covariance matrix, it then estimates the actual error essentially from the scatter in the data. If, however, your data points come with (reliable) error estimates, that actually throws away a lot of information, and the covariance matrix is less expressive than it could be.  To make ``curve_fit`` treat its sigma argument as actual error estimates and use their absolute (rather than relative, as with weights) values in pcov calculation, just pass ``absolute_sigma=True``.

## Exercise 1

In the following code, we generate some random data points:

In [ ]:
x = np.random.uniform(0., 10., 100)
y = np.polyval([1, 2, -3], x) + np.random.normal(0., 10., 100)
e = np.random.uniform(5, 10, 100)


Fit a line and a parabola to it and overplot the two models on top of the data:

In [ ]:
def line(x,a,b):
    return a*x+b

In [ ]:
def parabola(x,a0,a1,a2):
    return a0*x**2+a1*x+a2

In [ ]:
# your solution here
plt.figure(figsize=(8,6))
plt.errorbar(x,y,yerr=e,fmt='none')

p_line, cov_line =curve_fit(line,x,y,sigma=e)
plt.plot(x,line(x,p_line[0],p_line[1]),'g-')

p_par, cov_par =curve_fit(parabola,x,y,sigma=e)


plt.plot(x,parabola(x,p_par[0],p_par[1],p_par[2]),'ro')

plt.xlim(0,10)
plt.ylim(-25,125)

print(p_line,p_par)


## Exercise 2

As before, we use the [data/munich_temperatures_average_with_bad_data.txt](data/munich_temperatures_average_with_bad_data.txt) file, which gives the temperature in Munich every day for several years:

In [ ]:
# The following code reads in the file and removes bad values
import numpy as np
date, temperature = np.loadtxt('data/munich_temperatures_average_with_bad_data.txt', unpack=True)
keep = np.abs(temperature) < 90
date = date[keep]
temperature = temperature[keep]
plt.figure(figsize=(15,6))
plt.plot(date,temperature)
print(date)

Fit the following function to the data:

$$f(t) = a~\cos{(2\pi t + b)} + c$$

where $t$ is the time in years. Make a plot of the data and the best-fit model in the range 2008 to 2012. What are the best-fit values of the parameters? What is the overall average temperature in Munich, and what are the typical daily average values predicted by the model for the coldest and hottest time of year? What is the meaning of the ``b`` parameter, and does its value make sense?

In [ ]:
# your solution here
def fit_temperature(x,a,b,c):
    return a*np.cos(2*np.pi*x+b) +c
    

In [ ]:
par, cov = curve_fit(fit_temperature,date,temperature)
print(par)

In [ ]:
plt.figure(figsize=(15,6))
plt.plot(date,temperature,label='data')
plt.plot(date,fit_temperature(date,*par),'red',lw=4,label='fit')
plt.xlim(2008,2012)
plt.axhline(y=0,color='k')
plt.axhline(y=20,color='k')

plt.fill_between(date,5,15,color='k',alpha=0.2,label='uncertainty around mean')
plt.legend()

In [ ]:
print("temperature =",par[0],"*cos(2*pi*x+",par[1],")+",par[2],")")

# Introduction to Scipy: Interpolation and Integration

In this section, we will look at two other common sub-packages of Scipy: [scipy.interpolate](http://docs.scipy.org/doc/scipy/reference/interpolate.html) and [scipy.integrate](http://docs.scipy.org/doc/scipy/reference/integrate.html).

## Interpolation

The simplest interpolation routine in [scipy.interpolate](http://docs.scipy.org/doc/scipy/reference/interpolate.html) is [interp1d](http://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp1d.html#scipy.interpolate.interp1d):

In [ ]:
from scipy.interpolate import interp1d
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline

If we create a fake dataset:

In [ ]:
x = np.array([0., 1., 3., 4.])
y = np.array([0., 4., 2.7, 2.08])
plt.plot(x,y,'o-')

we can interpolate linearly by first creating an interpolating function:

In [ ]:
f = interp1d(x,y)

and we can then interpolate to any value of x within the original bounds:

In [ ]:
f(0.5)

In [ ]:
f(3.3)

In [ ]:
plt.plot(x,y,'o-')
plt.plot(0.5,f(0.5),'v')
plt.plot(3.3,f(3.3),'s')

It is also possible to interpolate to several values at the same time:

In [ ]:
f(np.array([0.5, 1.5, 2.5, 3.5]))

If the interpolating function is called outside the original range, an error is raised:

You can change this behavior by telling ``interp1d`` to not give an error in this case, but to use a set value:

In [ ]:
f(10)

In [ ]:
f = interp1d(x, y, bounds_error=False, fill_value=-10.)

In [ ]:
f(-1.0)

In [ ]:
f(np.array([-1., 1., 3., 6.]))

x = np.linspace(-5,8,50)
y = f(x)

plt.plot(x,y,'o')


By default, ``interp1d`` uses linear interpolation, but it is also possible to use e.g. cubic **spline** interpolation:

In [ ]:
x = np.array([0., 1., 3., 4.])
y = np.array([0., 4., 2.7, 2.08])
f = interp1d(x, y, kind='cubic')
f(0.5)
plt.plot(x,y,'s-')



xx = np.linspace(0,4.,100)
yy = f(xx)
plt.plot(xx,yy,'o')



Let's compare a few ways to interpolate:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
xp=np.arange(0,4,0.1)
f = interp1d(x,y)
g = interp1d(x,y,kind=2) # equivalent to 'quadratic'
h = interp1d(x,y,kind=3) # equivalent to 'cubic'

plt.plot(xp,f(xp),xp,g(xp),xp,h(xp))
plt.xlim(0,6)
plt.legend(['linear','quadratic spline','cubic spline'])

In [ ]:
def orig(x):
    return 3.11-3.11*(x-1)*(x-1)+0.5*x+0.39*x*x*x

xp=np.arange(0,4,0.5)
print (xp)
print (orig(xp))
print (h(xp))

For more information, see the documentation for [interp1d](http://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp1d.html#scipy.interpolate.interp1d). There are also other interpolation functions available (for example for spline interpolation), which you can read up about at [scipy.interpolate](http://docs.scipy.org/doc/scipy/reference/interpolate.html). 

[Wikipedia](https://en.wikipedia.org/wiki/Spline_%28mathematics%29) also has some information on splines.

## Integration

The available integration functions are listed at [scipy.integrate](http://docs.scipy.org/doc/scipy/reference/integrate.html#module-scipy.integrate). See also the tutorial [here](http://docs.scipy.org/doc/scipy/reference/tutorial/integrate.html). You will notice there are two kinds of functions - those that integrate actual Python functions, and those that integrate numerical functions defined by Numpy arrays.

First, we can take a look at one of the functions that can integrate actual Python functions. If we define a function:

In [ ]:
def simple_function(x):
    return 3. * x**2 + 2. * x + 1.

we can integrate it between limits using:

In [ ]:
from scipy.integrate import quad
print(quad(simple_function, 1., 2.))

As described in the documentation for [quad](http://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.quad.html#scipy.integrate.quad), the first value returned is the integral, and the second is the error on the integral. If we had solved the integral analytically, we would expect 11, so the result is correct. The names comes from quadrature for working out the area under curve by splitting it up into know sub-areas.

We can also define functions as Numpy arrays:

In [ ]:
x = np.linspace(1., 2., 1000)
y = 3. * x**2 + 2. * x + 1.

And in this case we can use for example the [simps](http://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.simps.html#scipy.integrate.simps) function to integrate using Simpson's rule:

In [ ]:
from scipy.integrate import simps
print(simps(y, x=x))

This can be very useful in cases where one wants to integrate actual data that cannot be represented as a simple function or when the function is only available numerically.

Note that there is an issue on the [scipy.integrate](http://docs.scipy.org/doc/scipy/reference/integrate.html#module-scipy.integrate) page - there should also be a menton of the ``trapz`` function which works similarly to ``simps`` but does trapezium integration:

In [ ]:
from scipy.integrate import trapz
print(trapz(y, x=x))

## Exercise 1

a) Write a function that takes ``x``, and the parameters for a Gaussian (amplitude, displacement, width) and returns the value of the Gaussian at ``x``:

In [ ]:
# your solution here


b) Use ``quad`` to compute the integral and compare to what you would expect.

In [ ]:
# your solution here


c) Now create two arrays ``x`` and ``y`` that contain the Gaussian for fixed values ``x``, and try and compute the integral using ``simps``.

In [ ]:
# your solution here


Compare this to what you found with ``quad`` and analytically.

## Differential equations

An important task in scientific computing is to solve differential equations.

*scipy.integrate* also has *odeint* which we use here to solve the damped oscillation

$$\ddot{x} + 2 \gamma \dot{x} + k x = 0$$


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from scipy.integrate import odeint

For the *odeint* solver, the differential equation needs to be split up into a system of first-order equations:

(1)    $\dot{x} = v$

(2)    $\dot{v} = -k x - 2 g v$

In [ ]:
def diffeq(x, t,k,g):

    #define/clear derivative
    dxdt = np.zeros(2)

    """ differential equation for damped oscillation
        split into first-order equation 
         x. = v
         v. = - k x - 2 gamma v
    """
    dxdt[0] = x[1]
    dxdt[1] = -k*x[0] - 2* g*x[1]

    return dxdt

""" constants """
k=0.1
g=0.25

x    = np.array([2.0,-0.8])         # initial position
time = np.linspace(0.0,100.0,100)   # evaluation times

""" calling the solver """
solution = odeint(diffeq, x, time, args=(k,g))

This is the corresponding analytical solution:

In [ ]:
def damped_oscillation(t,x,k,g):
    om=np.sqrt(k-g**2)
    
    A=(x[1]+g*x[0])/om
    B=x[0]
    
    return (A*np.sin(om*t)+B*np.cos(om*t))*np.exp(-g*t)

In [ ]:

plt.plot(time,solution[:,0],time,damped_oscillation(time,x,k,g))
plt.legend(['Numerical','Analytical'])

## Exercise 2

a) Plot the motion of a critically damped oscillator $\gamma$ = $2 \sqrt{k}$ 

In [ ]:

# solution here


b) Extend the differential equation by a forcing $f(x)=A_0 \cos (\omega_0 t)$ [choose $A_0$ and $\omega_0$].
Consider the initial behaviour of the system.

In [ ]:

# solution here
